## Setting up and running optimization in Jupyter


In [ ]:
from geecs_scanner.logging_setup import ensure_logging
import logging

ensure_logging(
    level=logging.INFO,     # or INFO
    console=True,
    log_dir=None,            # or a path if you want a file
    max_bytes=10_000_000,
    backup_count=5,
)

from geecs_scanner.data_acquisition.types import ScanConfig, ScanMode
# from geecs_data_utils.types import ScanConfig, ScanMode
from geecs_scanner.data_acquisition.default_scan_manager import get_default_scan_manager
from geecs_scanner.utils import ApplicationPaths
from pathlib import Path

from geecs_data_utils.scan_data import ScanData, ScanPaths
from image_analysis.config_loader import set_config_base_dir


set_config_base_dir(ScanPaths.paths_config.image_analysis_configs_path)

logging.getLogger("data_logger").setLevel(logging.WARNING)
logging.getLogger("geecs_scanner").setLevel(logging.WARNING)
logging.getLogger("geecs_python_api").setLevel(logging.WARNING)
logging.getLogger("image_analysis").setLevel(logging.WARNING)

# load the default experiment configuration for Undulator (shot control, etc.)
manager = get_default_scan_manager("Undulator")
manager.shot_control_variables['Trigger.Source']['OFF'] = "Single shot"
manager.shot_control_variables['Trigger.Source']['SCAN'] = "Internal"

app_paths = ApplicationPaths(experiment='Undulator')

# Define path to the optimizer config. See documentation for optimizer config structure
optimizer_config_path = (
   app_paths.exp_optimization_routines
    / "bax_alignment_example.yaml"
)

In [ ]:
# success = manager.reinitialize(config_path=config_filename)
success = manager.reinitialize()

scan_config = ScanConfig
scan_config.start = 1
scan_config.end = 8
scan_config.step = 1
scan_config.wait_time = 2
scan_config.scan_mode = ScanMode.OPTIMIZATION
scan_config.optimizer_config_path = optimizer_config_path

In [ ]:
manager.scan_config = scan_config
manager.start_scan_thread(scan_config)

In [ ]:
manager.optimizer.xopt.generator